# Lesson 05D - Multiple Kernel Size with RandomizedSearch


Most of the code here is based on that [post](file:///C:/Users/ishay/Documents/Data%20Science/Coursera%20and%20practice/ML_Mastery/NLP/Model%20Improvement%20and%20Best%20Practices/How%20to%20Develop%20an%20N-gram%20Multichannel%20Convolutional%20Neural%20Network%20for%20Sentiment%20Analysis.html) from ML Mastery.

In [1]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
#from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.layers import Convolution1D
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import MaxPooling1D
from keras.layers import Input
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [2]:
max_features = 5000

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features) # instead on nb_words=

17465344/17464789 [==============================] - 3s 0us/step


In [4]:
X_train.shape

(25000,)

**Sequence Padding**

In [5]:
maxlen = 400

We define maximum length of 400. It means that posts shoeter than 400 will be filled with 0s, and longer posts are cut.

In [6]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

**Now defining the model**

In [7]:
def create_model(kernel_size_conv1=3, n_strides=1, output_vec_dim=50):
    inputs_shape = Input(shape=(400,))
    # First channel
    #inputs_shape1 = Input(shape=(400,))
    embed1 = Embedding(input_dim=5000, output_dim=output_vec_dim)(inputs_shape)
    spat_drop1 = SpatialDropout1D(0.4)(embed1)
    conv1 = Conv1D(filters=250, kernel_size=kernel_size_conv1, padding='valid', activation='relu', strides=n_strides)(spat_drop1)
    pool1 = MaxPooling1D(pool_size=2)(conv1)
    flat1 = Flatten()(pool1)
    
    # Second channel
    #inputs_shape2 = Input(shape=(400,))
    embed2 = Embedding(input_dim=5000, output_dim=output_vec_dim)(inputs_shape)
    spat_drop2 = SpatialDropout1D(0.3)(embed2)
    conv2 = Conv1D(filters=250, kernel_size=4, padding='valid', activation='relu', strides=n_strides)(spat_drop2)
    pool2 = MaxPooling1D(pool_size=2)(conv2)
    flat2 = Flatten()(pool2)
    
    # Third channel
    #inputs_shape3 = Input(shape=(400,))
    embed3 = Embedding(input_dim=5000, output_dim=output_vec_dim)(inputs_shape)
    spat_drop3 = SpatialDropout1D(0.4)(embed3)
    conv3 = Conv1D(filters=250, kernel_size=5, padding='valid', activation='relu', strides=n_strides)(spat_drop3)
    pool3 = MaxPooling1D(pool_size=2)(conv3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # FC layers
    dense1 = Dense(250, activation='relu')(merged)
    drop1 = Dropout(0.3)(dense1)
    dense2 = Dense(20, activation='relu')(drop1)
    drop2 = Dropout(0.3)(dense2)
    output = Dense(1, activation='sigmoid')(drop2)
    model = Model(inputs=inputs_shape, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

In [9]:
model = KerasClassifier(build_fn=create_model, verbose=1, epochs=3, batch_size=32)

In [10]:
create_model().summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 50)      250000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 400, 50)      250000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 400, 50)      250000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dr

In [11]:
param_dist= {"kernel_size_conv1": [2,3],
             "n_strides": sp_randint(1,3),
             "output_vec_dim": sp_randint(60,100)}

In [12]:
random_grid = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter = 3, n_jobs=1, return_train_score=True, cv=2)
random_grid_result = random_grid.fit(X_train, y_train)

Epoch 1/3
12500/12500 [==============================] - 33s 3ms/step - loss: 0.6185 - acc: 0.6054
Epoch 2/3
12500/12500 [==============================] - 30s 2ms/step - loss: 0.3118 - acc: 0.8756
Epoch 3/3
12500/12500 [==============================] - 5s 398us/step
Epoch 1/3
12500/12500 [==============================] - 31s 2ms/step - loss: 0.6342 - acc: 0.5927
Epoch 2/3
12500/12500 [==============================] - 30s 2ms/step - loss: 0.3245 - acc: 0.8739
Epoch 3/3
12500/12500 [==============================] - 5s 399us/step
Epoch 1/3
12500/12500 [==============================] - 33s 3ms/step - loss: 0.6034 - acc: 0.6279
Epoch 2/3
12500/12500 [==============================] - 31s 3ms/step - loss: 0.2944 - acc: 0.8889
Epoch 3/3
12500/12500 [==============================] - 5s 431us/step
Epoch 1/3
12500/12500 [==============================] - 33s 3ms/step - loss: 0.6083 - acc: 0.6210
Epoch 2/3
12500/12500 [==============================] - 32s 3ms/step - loss: 0.2892 - acc: 0.

In [13]:
random_grid_result.best_score_

0.8735999999809265

In [14]:
random_grid_result.best_params_

{'kernel_size_conv1': 2, 'n_strides': 1, 'output_vec_dim': 70}

In [15]:
pd.DataFrame(random_grid_result.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel_size_conv1,param_n_strides,param_output_vec_dim,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,94.054661,0.926545,5.134947,0.028458,2,1,70,"{'kernel_size_conv1': 2, 'n_strides': 1, 'outp...",0.87352,0.87368,0.87360,0.00008,1,0.97608,0.97624,0.97616,0.00008
1,97.066740,0.002880,5.610691,0.024742,3,1,83,"{'kernel_size_conv1': 3, 'n_strides': 1, 'outp...",0.85832,0.88200,0.87016,0.01184,2,0.96688,0.98240,0.97464,0.00776
2,54.886357,0.048368,3.419538,0.036935,3,2,77,"{'kernel_size_conv1': 3, 'n_strides': 2, 'outp...",0.86456,0.86168,0.86312,0.00144,3,0.98360,0.98328,0.98344,0.00016
